In [ ]:
import pandas as pd
import re
import numpy as np

# Transformando os arquivos dbf em csv, unificando os arquivos por ano e criando novos csv para posterior analise no Tableau

O SUS disponibiliza os arquivos no formato `dbc` no site: https://datasus.saude.gov.br/transferencia-de-arquivos/ ao clicar em: 

SIM - Sistema de Informação de Mortalidade -> Dados -> DO - Declarações de Óbito - 1979 a 2021 -> Selecionar Ano desejado -> Selecionar estados -> Download -> arquivo.zip. 

Esse arquivo foi extraido e transformado em `dbf` utilizando o `Tabwin`, e posteriormente tratado aqui no python para transformação em `csv` para leitura no pandas e outras manipulações de dataframe.

Criando uma lista com nomes e paths dos arquivos para facilitar a transformação via loop:

In [ ]:
estados = sorted(["data/IBGE MORTALIDADE/todos/DOAC","data/IBGE MORTALIDADE/todos/DOAL","data/IBGE MORTALIDADE/todos/DOAM","data/IBGE MORTALIDADE/todos/DOAP","data/IBGE MORTALIDADE/todos/DOBA","data/IBGE MORTALIDADE/todos/DOBR","data/IBGE MORTALIDADE/todos/DOCE","data/IBGE MORTALIDADE/todos/DODF","data/IBGE MORTALIDADE/todos/DOES","data/IBGE MORTALIDADE/todos/DOGO","data/IBGE MORTALIDADE/todos/DOMA","data/IBGE MORTALIDADE/todos/DOMG","data/IBGE MORTALIDADE/todos/DOMS","data/IBGE MORTALIDADE/todos/DOMT","data/IBGE MORTALIDADE/todos/DOPA","data/IBGE MORTALIDADE/todos/DOPB","data/IBGE MORTALIDADE/todos/DOPE","data/IBGE MORTALIDADE/todos/DOPI","data/IBGE MORTALIDADE/todos/DOPR","data/IBGE MORTALIDADE/todos/DORJ","data/IBGE MORTALIDADE/todos/DORN","data/IBGE MORTALIDADE/todos/DORO","data/IBGE MORTALIDADE/todos/DORR","data/IBGE MORTALIDADE/todos/DORS","data/IBGE MORTALIDADE/todos/DOSC","data/IBGE MORTALIDADE/todos/DOSE","data/IBGE MORTALIDADE/todos/DOSP","data/IBGE MORTALIDADE/todos/DOTO"]*21)
#[[x,estados.count(x)] for x in set(estados)] #formula usada para verificar a quantidade e o que está na lista
anos = list(range(2000,2021))
anos = [str(x) for x in anos]
anos = [x+'.dbf' for x in anos]*28 # 28 pois existem 28 arquivos diferentes (existe um para Brasilia também)
arquivo = [x + y for x, y in zip(estados, anos)]

   Para transformar `dbf` em `csv`, primeiramente criei uma função usando a biblioteca `dbfpy`, utilizando a `dbf.Table` e `dbf.export`.
   
   O resultado foi satisfatório com alguns erros de codec já que os arquivos antes de 2018 são transformados usando o ascii e depois de 2018 são transformados usando o `asciii` (no caso do uso da dbfpy).
   
   Como a ideia desse projeto é de utilizar vários anos para a analise de uma forma mais pratica após mais pesquisa verifiquei que a função encontrada no link https://blog.finxter.com/how-to-convert-a-dbf-to-a-csv-in-python/ realiza a conversão sem usar a `dbfpy`. Por isso optei pelo uso da mesma (apesar que ligeiramente alterada), que não gerou erro na transformação, já que ele usa a biblioteca `dbfread` para ler o `dbf` e a biblioteca csv para converter arquivo lido no python do arquivo dbf em strings separadas por virgulas (arquivo csv).

In [ ]:
import csv
from dbfread import DBF
def dbf_to_csv(path):
    '''Converting the .dbf at the specified path and
       returns the name of the .csv file after conversion.
       The .csv file has same path and naming scheme. 
       Examples:
       (1) my_file.dbf --> my_file.csv
       (2) /path/to/file/my_file.dbf --> /path/to/file/my_file.csv
    '''
    # Set the name of the CSV file
    for path in arquivo:
        try:
            csv_path = path[:-4] + ".csv"
            # Create a DBF object, i.e., load the .dbf file into the code 
            dbf = DBF(path)
            # Create a CSV file and fill it with dbf data
            with open(csv_path, 'w', newline = '') as f:
                # Create the CSV writer
                writer = csv.writer(f)
                # Write the CSV column names
                writer.writerow(dbf.field_names)
                # Write the CSV rows
                for record in dbf:
                    writer.writerow(list(record.values()))
        except Exception as e:
            print(path)
            print(e)
            pass
    return csv_path
dbf_to_csv(arquivo)

Após analise preliminar dos bancos de dados, foi executada a concatenação dos bancos de dados por ano, para posterior analise no Tableau.
Para diminuição do tamanho do arquivo e efeitos de analise, foram selecionadas colunas especificas no `pd.concat` em que o número de nulos é baixo e também são relevantes para a analise, colunas duplicadas foram dropadas e depois transformados em `.csv`. A celula abaixo foi utilizada para os anos de 2014 a 2020, que contem as mesmas colunas em todos os estados.

In [ ]:
import os
ano = '2020'
csvs = [pd.read_csv(f'data/IBGE MORTALIDADE/todos/{arquivo}',usecols=['ORIGEM','DIFDATA','STDONOVA','STDOEPIDEM','DTRECORIGA','ATESTADO','CODIFICADO','TIPOBITO','DTCADASTRO','CODMUNOCOR','LOCOCOR','CODMUNRES','CONTADOR','IDADE','SEXO','CAUSABAS','DTOBITO','NUMEROLOTE','STCODIFICA','VERSAOSCB','DTRECEBIM','CAUSABAS_O','DTNASC','DTATESTADO','HORAOBITO','LINHAA','NATURAL','CODMUNNATU','RACACOR','ESTCIV','ESC','ESCFALAGR1','ESC2010']) for arquivo in os.listdir(r'data/IBGE MORTALIDADE/todos') if arquivo.endswith(f"{ano}.csv")]
csvs = pd.concat(csvs)
csvs.drop_duplicates(inplace=True) #dropando possiveis linhas duplicadas nos datasets
csvs.to_csv(f"data/IBGE MORTALIDADE/csvs{ano}.csv")

In [ ]:
municipios = pd.read_excel(r'data\MUNICIPIOS IBGE\RELATORIO_DTB_BRASIL_MUNICIPIO.xls')
df = pd.read_csv(r'data/IBGE MORTALIDADE/csvs2020.CSV')

Os campos de localidade estão preenchidos como codigo do IBGE, por isso foi feito o download da planilha de municipios e a adição dos nomes de municipios para as informações das colunas `'CODMUNOCOR'`, `'CODMUNNATU'` e `'CODMUNRES'`. As novas colunas foram posteriormente renomeadas e tratadas para refletir a informação de forma correta.

In [ ]:
municipios['Código Município Completo'] = municipios['Código Município Completo'].astype('str').str[:-1].astype('float')
#merge por CODMUNOCOR
df['CODMUNOCOR'] = df['CODMUNOCOR'].astype('float')
df = df.merge(municipios[['Código Município Completo','Nome_Município','Nome_UF']],left_on='CODMUNOCOR',right_on='Código Município Completo')
df.drop(columns='Código Município Completo',inplace=True)
df.rename(columns = {'Nome_Município' : 'MUNICIPIO_OCOR','Nome_UF':'UF_OCOR'},inplace=True)
df['MUNICIPIO_OCOR'] != df['MUNICIPIO_OCOR']
#merge por CODMUNNATU
df['CODMUNNATU'] = df['CODMUNNATU'].astype('float')
df = df.merge(municipios[['Código Município Completo','Nome_Município','Nome_UF']],left_on='CODMUNNATU',right_on='Código Município Completo',how='left')
df.drop(columns='Código Município Completo',inplace=True)
df.rename(columns = {'Nome_Município' : 'MUNICIPIO_NATU','Nome_UF':'UF_NATU'},inplace=True)
df.loc[df['MUNICIPIO_NATU'].isnull(),'MUNICIPIO_NATU'] = 'Estrangeiro'
df.loc[df['UF_NATU'].isnull(),'UF_NATU'] = 'Estrangeiro'
#merge por CODMUNRES
df['CODMUNRES'] = df['CODMUNRES'].astype('float')
df = df.merge(municipios[['Código Município Completo','Nome_Município','Nome_UF']],left_on='CODMUNRES',right_on='Código Município Completo',how='left')
df.drop(columns='Código Município Completo',inplace=True)
df.rename(columns = {'Nome_Município' : 'MUNICIPIO_RES','Nome_UF':'UF_RES'},inplace=True)

Foi descoberta a necessidade de criar novas planilhas `'.csv'` para integração com planilhas de CID existentes no SUS no `Tableau` ou `SQL`.

In [ ]:
#Tratando e criando um novo arquivo para posterior adição ao Tableau, para analisar os CID'S individualmente
df['ATESTADO'] = df['ATESTADO'].str.findall('[a-zA-Z]+\d+') #separando os cids do atestado em lista dentro da coluna
df['LINHAA'] = df['LINHAA'].str.findall('[a-zA-Z]+\d+')

ano = '2020'
dfcont = df[['CONTADOR','ATESTADO']]
dfcont = dfcont.explode('ATESTADO')
dfcont.to_csv(f'data/IBGE MORTALIDADE/dfcont{ano}.csv')

dfcont = df[['CONTADOR','LINHAA']] 
dfcont = dfcont.explode('LINHAA')
dfcont.to_csv(f'data/IBGE MORTALIDADE/dflinhaa{ano}.csv')

# Tratamento de Nulos e necessidade de colunas

Esse tratamento é feito por base de analise da quantidade de nulos em cada coluna no dataset original sem retirada de colunas. Também é feita a analise subjetiva de quais colunas são necessárias para a analise, sendo que as irrelevantes são retiradas do dataframe.

In [ ]:
pd.DataFrame(data=((df.isnull().sum())/(df.shape[0])).sort_values().head(44)).reset_index() #vendo a % de nulos nas colunas

In [ ]:
pd.DataFrame(data=((df.isnull().sum())/(df.shape[0])).sort_values().tail(44)).reset_index()
#vendo a % de nulos nas colunas

In [ ]:
#colunas_nona = list(pd.DataFrame(data=((df.isnull().sum())/(df.shape[0])).sort_values().head(44)).reset_index()['index'])[:40]
colunas_nona = ['DIFDATA','STDOEPIDEM','ATESTADO','CODIFICADO','DTCADASTRO','CODMUNOCOR','LOCOCOR','CODMUNRES','CONTADOR','IDADE','SEXO','CAUSABAS','DTOBITO','CAUSABAS_O','DTNASC','DTATESTADO','HORAOBITO','LINHAA','NATURAL','CODMUNNATU','RACACOR','ESTCIV','ESC','ESCFALAGR1','ESC2010','MUNICIPIO_OCOR','UF_OCOR','MUNICIPIO_NATU','UF_NATU','MUNICIPIO_RES','UF_RES']
#colocando o nome das colunas para efeito de tratamento em outros arquivos, para sempre manter as mesmas colunas e reportar erro em caso de colunas distintas
df = df[colunas_nona] #dropando colunas com muitos nulos
df.drop_duplicates(inplace=True) #dropando possiveis linhas duplicadas nos datasets


In [ ]:
ano = '2020'
dfcont = df[['CONTADOR','LINHAA']] 
dfcont = dfcont.explode('LINHAA')
dfcont.to_csv(f'data/IBGE MORTALIDADE/dflinhaa{ano}.csv')

In [ ]:
pd.DataFrame(data=((df.isnull().sum())/(df.shape[0])).sort_values()).reset_index()
#verificando novamente os nulos.

# Tratanto coluna de data de obitos

Para tratar a coluna de data de obitos, é necessário usar o pd.to_datetime, mas essa função não da certo pois algumas datas não tem a mesma quantidade de caracteres que outras (as datas de 1 a 10 não tem o zero na frente), por isso precisamos adicionar o zero na frete.

In [ ]:
df['DTOBITO'] = df['DTOBITO'].astype('str')
df['DTOBITO'] = df['DTOBITO'].str.zfill(8)
df['DTOBITO'] = pd.to_datetime(df['DTOBITO'],format='%d%m%Y')

# Padronizando a coluna de hora de obito

Nesse caso a hora de óbito possuia valores nulos, como isso não afeta a analise proposta, a linha será mantida e o tratamento será feito somente nas linhas que não possuem valor nulo, para ficar no formato hh:mm.

In [ ]:
df.loc[~df['HORAOBITO'].isnull(),'HORAOBITO'] = df['HORAOBITO'].astype('str').str.strip('0').str.strip('.').str.zfill(4)
df['HORAOBITO'] = (df['HORAOBITO'].str[:2]+':'+df['HORAOBITO'].str[2:4])

# Tratando a coluna Natural

Essa coluna fala da naturalidade, em que as que começam com 8 são do brasil e as outras são de estrangeiros. Como não precisamos saber de qual pais os estrangeiros vem, todos serão substituidos por 0 (novo codigo para estrangeiros).

In [ ]:
df['NATURAL'] = df['NATURAL'].astype('str')
df['NATURAL'] = df['NATURAL'].str.replace('nan','0')
estrangeiros = (~df['NATURAL'].str.startswith('8')&~df['NATURAL'].str.startswith('0'))
df['NATURAL'][estrangeiros] = '0'
df['NATURAL'] = df['NATURAL'].astype('float').astype('int')

# Tratando a coluna DTNASC

Devemos tratar e transformar para datetime, mas podemos ver que a coluna tem vários nulos e, que quando nessa coluna o valor é nulo, a maioria das outras colunas na mesma linha também ficam como nulos, por esse motivo foi decidido que as linhas com nulo na data de nascimento serão dropadas.

O tratamento que foi necessário após a exclusão foi retirar o .0 do float, acrescentar o 0 no primeiro digito das strings que continham menos de 8 caracteres (mesmo tratamento da ultima coluna de data) e passar para datetime com o pd.to_datetime.

Foram descobertos erros de digitação nessa coluna, por isso no pd.to_datetime foi necessário entrar com o argumento coerce, para rodar a função mesmo com esses erros.

In [ ]:
df = df[~df['DTNASC'].isna()] #Removendo as linhas com nulos na coluna data de nascimento

In [ ]:
df['DTNASC'] = df['DTNASC'].astype('int').astype('str').str.zfill(8)
df['DTNASC'] = pd.to_datetime(df['DTNASC'], format='%d%m%Y',errors='coerce')

# Tratando coluna IDADE

Essa coluna está preenchida por codigos conforme abaixo:

Idade, composto de dois subcampos. O primeiro, de 1 dígito, indica a unidade da idade, conforme a tabela a seguir. O segundo, de dois dígitos, indica a quantidade de unidades:

0: Idade ignorada, o segundo subcampo e

1: Horas, o segundo subcampo varia de 01 a 23

2: Dias, o segundo subcampo varia de 01 a 29

3: Meses, o segundo subcampo varia de 01 a 11

4: Anos, o segundo subcampo varia de 00 a 99

5: Anos (mais de 100 anos), o segundo subcampo varia de 0 a 99,Exemplos:

000: Idade ignorada

020: 20 minutos

103: 3 horas

204: 4 dias

305: 5 meses

400: menor de 1 ano, mas não se sabe o numero de horas, dias ou meses

410: 10 anos

505: 105 anos 

A idade será alterada para a idade correta em anos, no caso de menor que 1 ano será ajustada para 'menor 1'.

In [ ]:
df.loc[df['IDADE'].astype('str').str.startswith('1'),'IDADECORRIGIDA'] = 0
df.loc[df['IDADE'].astype('str').str.startswith('2'),'IDADECORRIGIDA'] = 0
df.loc[df['IDADE'].astype('str').str.startswith('3'),'IDADECORRIGIDA'] = 0
df.loc[df['IDADE'].astype('str').str.startswith('4'),'IDADECORRIGIDA'] = df['IDADE'].astype('str').str[1:]
df.loc[df['IDADE'].astype('str').str.startswith('5'),'IDADECORRIGIDA'] = ('1'+ df['IDADE'].astype('str').str[1:])

# Tratando outras colunas de Datas

No caso do tratamento de outras colunas de data, foi feito o mesmo tratamento da coluna `DTNASC`, para as colunas `DTATESTADO` e `DTCADASTRO`.

In [ ]:
df.loc[~df['DTATESTADO'].isna(),'DTATESTADO'] = df['DTATESTADO'].astype('str').str.zfill(10).str[:-2]
df['DTATESTADO'] = pd.to_datetime(df['DTATESTADO'], format='%d%m%Y',errors='coerce')

In [ ]:
df.loc[~df['DTCADASTRO'].isna(),'DTCADASTRO'] = df['DTCADASTRO'].astype('str').str.zfill(10).str[:-2]
df['DTCADASTRO'] = pd.to_datetime(df['DTCADASTRO'], format='%d%m%Y',errors='coerce')

# Criando um arquivo csv com o tratamento concluído

In [ ]:
df.to_csv('data/IBGE MORTALIDADE/dftratado2020.csv')